# Exercícios — Notebook 4 (Memória)

Este notebook contém 3 questões para você implementar usando os conceitos de memória vistos no encontro sobre: curto/longo prazo, janelas de contexto e armazenamento externo.

Importante: As células de exercício estão com TODOs e sem "respostas prontas" impressas. Você deve completar o código antes de executar os testes (comentados).


In [ ]:
%pip install -q -r ../requirements.txt

In [ ]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
assert api_key, "GOOGLE_API_KEY ausente no .env."
model_name = os.getenv("MODEL_NAME", "gemini-2.0-flash")
llm = ChatGoogleGenerativeAI(model=model_name, google_api_key=api_key, streaming=False)
print("LLM inicializado.")


## Questão 1 — Memória de curto prazo com LCEL

Implemente uma cadeia com `RunnableWithMessageHistory` e `InMemoryChatMessageHistory` para manter o histórico por sesssão.

Tarefas:
- Criar `ChatPromptTemplate` com `MessagesPlaceholder("chat_history")`.
- Implementar `get_session_history(session_id)` que retorna um `InMemoryChatMessageHistory` por sessão.
- Encapsular a cadeia com `RunnableWithMessageHistory`.

Teste (após implementar):
- Use o mesmo `session_id` para duas chamadas e observe que a resposta considera o histórico.
- Mude o `session_id` e verifique que o contexto é novo.


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableWithMessageHistory

# TODO: Defina o prompt com chat_history
prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente. Mantenha o contexto."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

# TODO: Implementar armazenamento por sessão
store = {}
def get_session_history(session_id: str):
    # Implemente aqui (retorne InMemoryChatMessageHistory)
    raise NotImplementedError("Implemente get_session_history(session_id)")

# Cadeia base
chain = prompt | llm | StrOutputParser()

# TODO: Encapsular com RunnableWithMessageHistory
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# Testes (descomente após implementar)
# sidA = "A"
# print(chain_with_history.invoke({"input": "Oi, quem é você?"}, config={"configurable": {"session_id": sidA}}))
# print(chain_with_history.invoke({"input": "O que você lembra do que eu disse?"}, config={"configurable": {"session_id": sidA}}))
# sidB = "B"
# print(chain_with_history.invoke({"input": "Olá!"}, config={"configurable": {"session_id": sidB}}))


## Questão 2 — Janela de contexto (últimas k mensagens)

Implemente uma função que mantém apenas as últimas `k` mensagens por sessão para reduzir custo e focar no recente.

Tarefas:
- Implementar `get_session_history_k(session_id, k)` para devoler `InMemoryChatMessageHistory` e podar para as últimas `k` mensagens.
- Encapsular com `RunnableWithMessageHistory`.

Teste (após implementar):
- Faça 5 interações e verifique que apenas as últimas `k` são consideradas ao perguntar "O que você lembra?".


In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

store_k = {}
def get_session_history_k(session_id: str, k: int = 4):
    # Implemente aqui (retorne InMemoryChatMessageHistory e mantenha só últimas k)
    raise NotImplementedError("Implemente get_session_history_k(session_id, k)")

chain_window = RunnableWithMessageHistory(
    # Reaproveita a 'chain' da Questão 1
    chain,
    lambda sid: get_session_history_k(sid, k=4),
    input_messages_key="input",
    history_messages_key="chat_history",
)

# Teste (descomente após implementar)
# sidW = "W"
# for i in range(1, 6):
#     chain_window.invoke({"input": f"Msg {i}"}, config={"configurable": {"session_id": sidW}})
# print(chain_window.invoke({"input": "O que você lembra?"}, config={"configurable": {"session_id": sidW}}))


## Questão 3 — Armazenamento externo (SQLite)

Persista o histórico em um banco SQLite usando `SQLChatMessageHistory`.

Tarefas:
- Implementar `get_sql_history(session_id)` que retorna `SQLChatMessageHistory` com `connection_string="sqlite:///exercicios_notebooks3.db"`.
- Encapsular com `RunnableWithMessageHistory`.

Teste (após implementar):
- Reexecute a célula com o mesmo `session_id` e observe que a memória persiste no arquivo SQLite.


In [ ]:
%pip install -q sqlalchemy

In [ ]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

def get_sql_history(session_id: str):
    # Implemente aqui (retorne SQLChatMessageHistory apontando para 'sqlite:///exercicios_notebooks3.db')
    raise NotImplementedError("Implemente get_sql_history(session_id)")

chain_sql = RunnableWithMessageHistory(
    chain,
    get_sql_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# Teste (descomente após implementar)
# sidDB = "DB1"
# print(chain_sql.invoke({"input": "Olá, registrando no DB externo."}, config={"configurable": {"session_id": sidDB}}))
# print(chain_sql.invoke({"input": "O que você lembra desta sessão?"}, config={"configurable": {"session_id": sidDB}}))


## Critérios e Dicas

- Q1: respostas consideram histórico dentro da mesma sessão.
- Q2: contexto inclui apenas as últimas `k` mensagens.
- Q3: com mesmo `session_id`, histórico persiste em `exercicios_notebooks3.db`.

Dicas:
- Use o mesmo `session_id` para manter o contexto; mude para conversas novas.
- Combine janela de contexto com sumarização (do encontro de memória) para históricos longos.
- Garanta que o caminho SQLite esteja acessível no seu ambiente.
